## Feature Engineering
This section takes pre-engineered lookup files and creates new features in the training data. It also transposes previous weeks' sales transactions to act as features for current week data

In [1]:
# Imports and Settings

# specify a data file path
data_location = "../data/"

# imports for data exploration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# further imports to go here
import time

### Load training data

In [2]:
# let's now take in the smaller training data file
train_file = "smaller_train.csv"

# specify columns for the training data as spanish is not a strong point in London
train_columns = ['WeekNumber','AgencyId','ChannelId','RouteId','ClientId','ProductId','Sales',
                 'SalesPesos','Returns','ReturnsPesos','Demand']

# load the training data in full, skip the column headings on the first rows
t0 = time.time() 
df_train = pd.read_csv(data_location + train_file,names=train_columns,skiprows=1)
print 'loading complete in: ', time.time()-t0, ' seconds'

loading complete in:  1.27300000191  seconds


### Create lag features
We will create lagged sales data for 3 previous periods which will mean we will not have target labels for weeks 3,4 or 5

In [3]:
# first separate the data into week by week dataframes
min_week = df_train["WeekNumber"].min()
run_week = min_week
max_week = df_train["WeekNumber"].max()

t0 = time.time()

filtered_data = {}
while (run_week <= max_week):
    # slice training data by week
    df_filter = df_train[df_train['WeekNumber']==run_week]
    
    # store in dictionary against run week
    filtered_data[run_week] = df_filter
    run_week += 1

t1 = time.time()
print 'data separated into weeks in: ', t1-t0, ' seconds'

# now need to create a new training set with lag weeks and tidy up
run_week = max_week
joined_data = []
while (run_week-3 >= min_week):
    # create lagged data from -1,-2,-3 for 0
    
    df = filtered_data[run_week]
    
    #lag 1
    df_1 = filtered_data[run_week-1]
    df_1 = df_1.drop('WeekNumber',axis=1)
    filter_1 = ['AgencyId','ChannelId','RouteId','ClientId','ProductId','Sales_L1','SalesPesos_L1','Returns_L1',
                'ReturnsPesos_L1','Demand_L1']
    df_1.columns = filter_1
    
    #lag 2
    df_2 = filtered_data[run_week-2]
    df_2 = df_2.drop('WeekNumber',axis=1)
    filter_2 = ['AgencyId','ChannelId','RouteId','ClientId','ProductId','Sales_L2','SalesPesos_L2','Returns_L2',
                'ReturnsPesos_L2','Demand_L2']
    df_2.columns = filter_2
    
    df_3 = filtered_data[run_week-3]
    df_3 = df_3.drop('WeekNumber',axis=1)
    filter_3 = ['AgencyId','ChannelId','RouteId','ClientId','ProductId','Sales_L3','SalesPesos_L3','Returns_L3',
                'ReturnsPesos_L3','Demand_L3']
    df_3.columns = filter_3
    
    # merge the frames...
    join_keys = ['AgencyId','ChannelId','RouteId','ClientId','ProductId']
    df = pd.merge(df, df_1, how='outer', on=join_keys)
    df = pd.merge(df, df_2, how='outer', on=join_keys)
    df = pd.merge(df, df_3, how='outer', on=join_keys)
    joined_data.append(df)
    run_week -= 1

t2 = time.time()
print 'lagged data created in: ', t2-t1, ' seconds'

# now merge joined_data into a single frame - df_train...
for i, item in enumerate(joined_data):
    if i == 0:
        df_train = joined_data[i]
    else:
        df_train = pd.concat([df_train,joined_data[i]],ignore_index=True)

print 'loading complete in: ', time.time()-t0, ' seconds. Dataset now has:', len(df_train.index), ' records'

data separated into weeks in:  0.118999958038  seconds
lagged data created in:  3.87400007248  seconds
loading complete in:  4.26600003242  seconds. Dataset now has: 1062620  records


### Add in Lookup Data
Link to Client, Product and Agency to add additional potential features

In [4]:
# load the lookup files
t0 = time.time()
product_lookup = "engineered_producto_tabla.csv"
product_columns = ['ProductId','OriginalName','ProductName','ProductPieces','ProductWeight','SupplierCode']
df_product = pd.read_csv(data_location + product_lookup,names=product_columns,skiprows=1)

client_lookup = "engineered_cliente_tabla.csv"
client_columns = ['ClientId','OriginalName','ClientName']
df_client = pd.read_csv(data_location + client_lookup,names=client_columns,skiprows=1)

agency_lookup = "town_state.csv"
agency_columns = ['AgencyId','Town','State']
df_agency = pd.read_csv(data_location + agency_lookup,names=agency_columns,skiprows=1)

print 'loading complete in: ', time.time()-t0, ' seconds'

loading complete in:  0.40900015831  seconds


In [5]:
# now we need to join the 3 engineered lookup tables to the training data...
t0 = time.time()

# let's join to the agency table to add in some features
df_train = pd.merge(left=df_agency,right=df_train, how='right', left_on='AgencyId', right_on='AgencyId')

# let's join to the product table on ProductId
df_train = pd.merge(left=df_product,right=df_train, how='right', left_on='ProductId', right_on='ProductId')

# let's join to the client table on ClientId
df_train = pd.merge(left=df_client,right=df_train, how='right', left_on='ClientId', right_on='ClientId')

print 'merge complete in: ', time.time()-t0, ' seconds'

merge complete in:  3.44099998474  seconds


In [6]:
# we'll lose the original name columns as they're not useful for us
df_train = df_train.drop(['OriginalName_x','OriginalName_y'],axis=1)
print df_train.columns.values

['ClientId' 'ClientName' 'ProductId' 'ProductName' 'ProductPieces'
 'ProductWeight' 'SupplierCode' 'AgencyId' 'Town' 'State' 'WeekNumber'
 'ChannelId' 'RouteId' 'Sales' 'SalesPesos' 'Returns' 'ReturnsPesos'
 'Demand' 'Sales_L1' 'SalesPesos_L1' 'Returns_L1' 'ReturnsPesos_L1'
 'Demand_L1' 'Sales_L2' 'SalesPesos_L2' 'Returns_L2' 'ReturnsPesos_L2'
 'Demand_L2' 'Sales_L3' 'SalesPesos_L3' 'Returns_L3' 'ReturnsPesos_L3'
 'Demand_L3']


In [7]:
# now write out the engineered training file to disk
df_train.to_csv(data_location + 'engineered_train.csv',index=False)

In [8]:
# let's load it back in as a test...
train_file = "engineered_train.csv"

# specify columns for the training data as spanish is not a strong point in London
train_columns = ['ClientId','ClientName','ProductId','ProductName','ProductPieces','ProductWeight'
                 ,'SupplierCode','AgencyId','Town','State','WeekNumber','ChannelId','RouteId'
                 ,'Sales','SalesPesos','Returns','ReturnsPesos','Demand','Sales_L1','SalesPesos_L1'
                 ,'Returns_L1','ReturnsPesos_L1','Demand_L1','Sales_L2','SalesPesos_L2','Returns_L2','ReturnsPesos_L2'
                 ,'Demand_L2','Sales_L3','SalesPesos_L3','Returns_L3','ReturnsPesos_L3','Demand_L3']

# load the training data in full, skip the column headings on the first rows
t0 = time.time() 
df_train = pd.read_csv(data_location + train_file,names=train_columns,skiprows=1)
print 'loading complete in: ', time.time()-t0, ' seconds'

loading complete in:  5.5759999752  seconds


In [9]:
print df_train.head()

   ClientId ClientName  ProductId ProductName ProductPieces ProductWeight  \
0       215   FACULTAD       1240  Mantecadas           4p          125g    
1       215   FACULTAD       1240  Mantecadas           4p          125g    
2       215   FACULTAD       1240  Mantecadas           4p          125g    
3       215   FACULTAD       1242     Donitas           6p          105g    
4       215   FACULTAD       1242     Donitas           6p          105g    

  SupplierCode  AgencyId                    Town             State    ...      \
0          BIM      2095  2175 TOLUCA AEROPUERTO  ESTADO DE MÉXICO    ...       
1          BIM      2095  2175 TOLUCA AEROPUERTO  ESTADO DE MÉXICO    ...       
2          BIM      2095  2175 TOLUCA AEROPUERTO  ESTADO DE MÉXICO    ...       
3          BIM      2095  2175 TOLUCA AEROPUERTO  ESTADO DE MÉXICO    ...       
4          BIM      2095  2175 TOLUCA AEROPUERTO  ESTADO DE MÉXICO    ...       

   Sales_L2  SalesPesos_L2  Returns_L2  ReturnsPes